In [2]:
import numpy as np 
import pandas as pd
import tensorflow
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.metrics import log_loss
import tensorflow as tf

In [3]:
data_train = pd.read_csv('train_features.csv')
data_test = pd.read_csv('test_features.csv')
data_train_target_ns = pd.read_csv('train_targets_nonscored.csv')
data_train_target_s = pd.read_csv('train_targets_scored.csv')
sub = pd.read_csv('sample_submission.csv')

In [4]:
display(data_train.head())
print("SHAPE of training data--",data_train.shape)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


SHAPE of training data-- (23814, 876)


In [5]:
display(data_test.head())
print("SHAPE of test data--",data_test.shape)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


SHAPE of test data-- (3982, 876)


In [6]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72:2})
    del df['sig_id']
    return df

train = preprocess(data_train)
test = preprocess(data_test)

del data_train_target_s['sig_id']

In [7]:
def create_model(num_columns):
    model = Sequential()
    model.add(Input(num_columns))
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(206, activation='sigmoid'))
    
    optimizer = tfa.optimizers.Lookahead('adam',sync_period=10)
    
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    model.summary()
    return model

In [8]:
def metric(y_true, y_pred):
    metrics = []
    for _target in data_train_target_s.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [10]:
N_STARTS = 2
tf.random.set_seed(42)

res = data_train_target_s.copy()
sub.loc[:, data_train_target_s.columns] = 0
sub.loc[:, data_train_target_s.columns] = 0

for seed in range(N_STARTS):
    for n, (train_idx, test_idx) in enumerate(KFold(n_splits=5, random_state=seed, shuffle=True).split(data_train_target_s, data_train_target_s)):
        print(f'Fold {n}')
    
        model = create_model(875)
        checkpoint_path = f'repeat:{seed}_Fold:{n}.h5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[train_idx],
                  data_train_target_s.values[train_idx],
                  validation_data=(train.values[test_idx], data_train_target_s.values[test_idx]),
                  epochs=25, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=1
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values)
        val_predict = model.predict(train.values[test_idx])
        
        sub.loc[:, data_train_target_s.columns] += test_predict
        res.loc[test_idx, data_train_target_s.columns] += val_predict
        print('')
    
sub.loc[:, data_train_target_s.columns] /= ((n+1) * N_STARTS)
res.loc[:, data_train_target_s.columns] /= N_STARTS

Fold 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 875)               3500      
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              1794048   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 206)       

Epoch 9/25
149/149 [==============================] - 26s 174ms/step - loss: 0.0186 - accuracy: 0.0938 - val_loss: 0.0177 - val_accuracy: 0.1136 - lr: 0.0010
Epoch 10/25
149/149 [==============================] - 30s 201ms/step - loss: 0.0183 - accuracy: 0.1011 - val_loss: 0.0176 - val_accuracy: 0.0991 - lr: 0.0010
Epoch 11/25
149/149 [==============================] - 23s 153ms/step - loss: 0.0180 - accuracy: 0.1059 - val_loss: 0.0173 - val_accuracy: 0.1136 - lr: 0.0010
Epoch 12/25
149/149 [==============================] - 30s 199ms/step - loss: 0.0174 - accuracy: 0.1125 - val_loss: 0.0169 - val_accuracy: 0.1260 - lr: 0.0010
Epoch 13/25
149/149 [==============================] - 28s 190ms/step - loss: 0.0172 - accuracy: 0.1170 - val_loss: 0.0169 - val_accuracy: 0.1247 - lr: 0.0010
Epoch 14/25
149/149 [==============================] - 25s 170ms/step - loss: 0.0169 - accuracy: 0.1183 - val_loss: 0.0167 - val_accuracy: 0.1054 - lr: 0.0010
Epoch 15/25
149/149 [==========================

149/149 [==============================] - 22s 144ms/step - loss: 0.0148 - accuracy: 0.1533 - val_loss: 0.0155 - val_accuracy: 0.1190 - lr: 1.0000e-04
Epoch 24/25
149/149 [==============================] - 21s 140ms/step - loss: 0.0148 - accuracy: 0.1527 - val_loss: 0.0155 - val_accuracy: 0.1207 - lr: 1.0000e-04
Epoch 25/25
149/149 [==============================] - 19s 127ms/step - loss: 0.0146 - accuracy: 0.1594 - val_loss: 0.0154 - val_accuracy: 0.1195 - lr: 1.0000e-04

Fold 3
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 875)               3500      
_________________________________________________________________
dense_12 (Dense)             (None, 2048)              1794048   
_________________________________________________________________
dropout_8 (Dropout)          (None, 2048)              0         
_______________________________

Epoch 6/25
149/149 [==============================] - 17s 116ms/step - loss: 0.0206 - accuracy: 0.0709 - val_loss: 0.0194 - val_accuracy: 0.0842 - lr: 0.0010
Epoch 7/25
149/149 [==============================] - 18s 118ms/step - loss: 0.0199 - accuracy: 0.0800 - val_loss: 0.0188 - val_accuracy: 0.1077 - lr: 0.0010
Epoch 8/25
149/149 [==============================] - 18s 118ms/step - loss: 0.0193 - accuracy: 0.0901 - val_loss: 0.0186 - val_accuracy: 0.1088 - lr: 0.0010
Epoch 9/25
149/149 [==============================] - 18s 118ms/step - loss: 0.0187 - accuracy: 0.0919 - val_loss: 0.0180 - val_accuracy: 0.1105 - lr: 0.0010
Epoch 10/25
149/149 [==============================] - 17s 117ms/step - loss: 0.0183 - accuracy: 0.0982 - val_loss: 0.0179 - val_accuracy: 0.1079 - lr: 0.0010
Epoch 11/25
149/149 [==============================] - 17s 116ms/step - loss: 0.0179 - accuracy: 0.1042 - val_loss: 0.0172 - val_accuracy: 0.0966 - lr: 0.0010
Epoch 12/25
149/149 [=============================

Epoch 23/25
149/149 [==============================] - ETA: 0s - loss: 0.0141 - accuracy: 0.1710
Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
149/149 [==============================] - 20s 134ms/step - loss: 0.0141 - accuracy: 0.1710 - val_loss: 0.0158 - val_accuracy: 0.1222 - lr: 0.0010
Epoch 24/25
149/149 [==============================] - 19s 124ms/step - loss: 0.0133 - accuracy: 0.1877 - val_loss: 0.0155 - val_accuracy: 0.1249 - lr: 1.0000e-04
Epoch 25/25
149/149 [==============================] - 18s 121ms/step - loss: 0.0131 - accuracy: 0.1939 - val_loss: 0.0155 - val_accuracy: 0.1281 - lr: 1.0000e-04

Fold 1
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_14 (Batc (None, 875)               3500      
_________________________________________________________________
dense_21 (Dense)             (None, 2048)              

Epoch 6/25
149/149 [==============================] - 20s 135ms/step - loss: 0.0210 - accuracy: 0.0664 - val_loss: 0.0192 - val_accuracy: 0.1031 - lr: 0.0010
Epoch 7/25
149/149 [==============================] - 21s 143ms/step - loss: 0.0200 - accuracy: 0.0787 - val_loss: 0.0187 - val_accuracy: 0.1144 - lr: 0.0010
Epoch 8/25
149/149 [==============================] - 20s 135ms/step - loss: 0.0196 - accuracy: 0.0880 - val_loss: 0.0183 - val_accuracy: 0.0852 - lr: 0.0010
Epoch 9/25
149/149 [==============================] - 20s 131ms/step - loss: 0.0189 - accuracy: 0.0914 - val_loss: 0.0177 - val_accuracy: 0.0934 - lr: 0.0010
Epoch 10/25
149/149 [==============================] - 22s 145ms/step - loss: 0.0186 - accuracy: 0.0982 - val_loss: 0.0173 - val_accuracy: 0.0970 - lr: 0.0010
Epoch 11/25
149/149 [==============================] - 19s 130ms/step - loss: 0.0179 - accuracy: 0.1029 - val_loss: 0.0171 - val_accuracy: 0.0945 - lr: 0.0010
Epoch 12/25
149/149 [=============================

Epoch 23/25
149/149 [==============================] - ETA: 0s - loss: 0.0141 - accuracy: 0.1748
Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
149/149 [==============================] - 21s 143ms/step - loss: 0.0141 - accuracy: 0.1748 - val_loss: 0.0158 - val_accuracy: 0.1262 - lr: 0.0010
Epoch 24/25
149/149 [==============================] - 18s 119ms/step - loss: 0.0132 - accuracy: 0.1926 - val_loss: 0.0157 - val_accuracy: 0.1291 - lr: 1.0000e-04
Epoch 25/25
149/149 [==============================] - 17s 117ms/step - loss: 0.0130 - accuracy: 0.1952 - val_loss: 0.0156 - val_accuracy: 0.1316 - lr: 1.0000e-04

Fold 4
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_20 (Batc (None, 875)               3500      
_________________________________________________________________
dense_30 (Dense)             (None, 2048)             

In [11]:
print(f'OOF Metric: {metric(data_train_target_s, res)}')

OOF Metric: 0.0037305020626018834


In [12]:
sub.loc[test['cp_type']==1, data_train_target_s.columns] = 0

In [13]:
sub.to_csv('submission.csv', index=False)